RNNs lose their effectiveness in most tasks by forgetting through long training time steps. However, Long Short-Term Memory networks (LSTMs) have great memories and can remember information which the vanilla RNN is unable to!

A significant shortcoming that plagues the typical RNN is the problem of vanishing/exploding gradients and it arises when back-propagating through the RNN during training, especially for networks with deeper layers. The gradients have to go through continuous matrix multiplications during the back-propagation process due to the chain rule, causing the gradient to either shrink exponentially (vanish) or blow up exponentially (explode). Having a gradient that is too small prevents the weights from updating and learning, whereas extremely large gradients cause the model to be unstable.

Due to these issues, RNNs are unable to work with longer sequences and hold on to long-term dependencies, making them suffer from “short-term memory”.

## What are LSTMs

While LSTMs are a kind of RNN and function similarly to traditional RNNs, its Gating mechanism is what sets it apart. This feature addresses the “short-term memory” problem of RNNs.

<img src='./imgs/rnn_lstm.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>

'Short-term memory' is especially important in the majority of Natural Language Processing (NLP) or time-series and sequential tasks. For example, let’s say we have a network generating text based on some input given to us. At the start of the text, it is mentioned that the author has a “**dog named Cliff**”. After a few other sentences where there is no mention of a pet or dog, the author brings up his pet again, and the model has to generate the next word to "However, Cliff, my pet $__$". As the word pet appeared right before the blank, a RNN can deduce that the next word will likely be an animal that can be kept as a pet.

<img src='./imgs/exam1.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>

However, due to the short-term memory, the typical RNN will only be able to use the contextual information from the text that appeared in the last few sentences - which is not useful at all. The RNN has no clue as to what animal the pet might be as the relevant information from the start of the text has already been lost.

On the other hand, the LSTM can retain the earlier information that the author has a pet dog, and this will aid the model in choosing "the dog" when it comes to generating the text at that point due to the contextual information from a much earlier time step.

<img src='./imgs/exam2.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>



### Inner workings of the LSTM

In the normal RNN cell, the input at a time-step and the hidden state from the previous time step is passed through a ***tanh*** activation function to obtain a new hidden state and output.

<img src='./imgs/inner_arch.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>



On the other hand, at each time step, the LSTM cell takes in 3 different pieces of information 

* the current input data
* the short-term memory from the previous cell (similar to hidden states in RNNs) 
* lastly the long-term memory

The short-term memory is commonly referred to as the hidden state, and the long-term memory is usually known as the cell state.

The cell then uses gates to regulate the information to be kept or discarded at each time step before passing on the long-term and short-term information to the next cell.

> How does the cell decide which information is usefull ?

These gates are called **the Input Gate, the Forget Gate, and the Output Gate**. 

<img src='./imgs/gates.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>

### Input Gate

<img src='./imgs/input_gate.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>

The input gate decides what new information will be stored in the long-term memory. It only works with the information from the current input and the short-term memory from the previous time step. 

Mathematically, this is achieved using 2 layers. The first layer can be seen as the filter which selects what information can pass through it and what information to be discarded. To create this layer, we pass the short-term memory and current input into a sigmoid function. The sigmoid function will transform the values to be between 0 and 1, with 0 indicating that part of the information is unimportant, whereas 1 indicates that the information will be used. As the layer is being trained through back-propagation, the weights in the sigmoid function will be updated such that it learns to only let the useful pass through while discarding the less critical features.

$$ i_1 = \alpha(W_{i_1}\times (H_{t-1}, x_t) + bias_{i_1})$$

> $(H_{t-1}, x_t)$ is concatenation?

The second layer takes the short term memory and current input as well and passes it through an activation function, usually the $tanh$ function, to regulate the network.

$$ i_2 = tanh(W_{i_2}\times (H_{t-1}, x_t) + bias_{i_2})$$



The outputs from these 2 layers are then multiplied, and the final outcome represents the information to be kept in the long-term memory and used as the output.

$$ i_{input} = i_1\times i_2 $$

### Forget Gate

<img src='./imgs/forget_gate.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>

The forget gate decides which information from the long-term memory should be kept or discarded. This is done by multiplying the incoming long-term memory by a forget vector generated by the current input and incoming short-term memory.

The forget vector($f$) is also a selective filter layer. To obtain the forget vector, the short-term memory, and current input is passed through a sigmoid function. The vector will be made up of 0s and 1s and will be multiplied with the long-term memory to choose which parts of the long-term memory to retain.

$$ f = \alpha(W_{forget}\times (H_{t-1}, x_t) + bias_{forget})$$


The outputs from the Input gate and the Forget gate will undergo a pointwise addition to give a new version of the long-term memory, which will be passed on to the next cell. This new long-term memory will also be used in the final gate, the Output gate.

$$ C_t = C_{t-1} \times f + i_{input}$$


### Output Gate

<img src='./imgs/output_gate.JPG'>
<center><span style="color:gray"><em>from https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/</em></span></center>

The output gate will take the current input, the previous short-term memory, and the newly computed long-term memory to produce the new short-term memory/hidden state which will be passed on to the cell in the next time step. The output of the current time step can also be drawn from this hidden state.

First, the previous short-term memory and current input will be passed into a sigmoid function to create the final filter. Then, we put the new long-term memory through an activation $tanh$ function. The output from these 2 processes will be multiplied to produce the new short-term memory.

$$ O_1 = \alpha(W_{output_1}\times (H_{t-1}, x_t) + bias_{output_1})$$


$$ O_2 = tanh(W_{output_2}\times C_t + bias_{output_2})$$

$$ H_t, O_t = O_1\times O_2$$